In [1]:
import sys
sys.path.insert(0, "..")
from gp3.inference import Vanilla
from gp3.utils import data as sim
from gp3.utils.structure import kron_list
from gp3.kernels import RBF
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
from plotly import tools
from IPython.display import display
init_notebook_mode(connected=True)
import warnings
warnings.filterwarnings('ignore')
from tqdm import trange
from ipywidgets import IntProgress
import numpy as np

In [2]:
X = sim.sim_X_equispaced(D = 2, N_dim = 30, lower=0, upper=100)
f = sim.sim_f(X, RBF(50., 1., 0.0), mu = 5.)
y = f + np.random.normal(size = len(f))

In [3]:
trace_func = go.Scatter3d(x = X[:,0], y = X[:,1], z=f, mode = 'markers', marker=dict(size = 2,))
trace_draws = go.Scatter3d(x = X[:,0], y = X[:,1], z=y, mode = 'markers', marker=dict(size = 2,))
fig = tools.make_subplots(rows=1, cols=2, specs=[[{'is_3d': True}, {'is_3d': True}]])
fig.append_trace(trace_func, 1, 1)
fig.append_trace(trace_draws, 1, 2)
iplot(fig)

This is the format of your plot grid:
[ (1,1) scene1 ]  [ (1,2) scene2 ]



In [4]:
X_part, y_part = sim.rand_partial_grid(X, y, 0.3)
X_full, y_full, obs_idx, imag_idx = sim.fill_grid(X_part, y_part)
color = np.zeros(X.shape[0])
color[obs_idx] = 1

inf = Vanilla(X_full, y_part, RBF(50., 1.), obs_idx = obs_idx, noise = 1., opt_kernel = True)

In [5]:
f_pred = inf.predict_mean()
iplot([go.Scatter3d(x = X[:,0], y = X[:,1], z=inf.predict_mean(), mode = 'markers', marker=dict(size = 2, color = color))])

In [6]:
var_est = inf.variance(500)
var_exact = inf.variance_exact()

In [7]:
trace_est = go.Scatter3d(x = X[:,0], y = X[:,1], z=var_est, mode = 'markers', marker=dict(size = 2, color = color))
trace_exact = go.Scatter3d(x = X[:,0], y = X[:,1], z=var_exact, mode = 'markers', marker=dict(size = 2, color = color))

fig = tools.make_subplots(rows=1, cols=2, specs=[[{'is_3d': True}, {'is_3d': True}]])
fig.append_trace(trace_est, 1, 1)
fig.append_trace(trace_exact, 1, 2)
iplot(fig)

This is the format of your plot grid:
[ (1,1) scene1 ]  [ (1,2) scene2 ]



In [8]:
inf.marginal()

-404.98902675446806

In [11]:
inf.grad_marginal()

[0.021282406776395178, 72.70485057163961, -4.0277597226437135e-05]